In [1]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt

## Helper functions

In [ ]:
def save_variable_to_file(name, filename):
    named_variable = [v for v in tf.trainable_variables() if v.name == name][0]    
    named_variable_np_array = named_variable.eval()
    
    with open(filename, "w") as outfile:
        np.save(outfile, named_variable_np_array)
    print('Variable: {} saved to file'.format(name))


In [5]:
model = 'log-uniform_clipped-alpha_MNIST'
model_path = '../../models/mnf/mnf_{}/'.format(model)

with tf.Session() as sess:
    saver = tf.train.import_meta_graph('{}mnf.meta'.format(model_path))
    saver.restore(sess, tf.train.latest_checkpoint(model_path))

    for v in tf.global_variables():
        print(v)

#     save_variable_to_file("fq2_fr2_usezTrue/densemnf_1/mean_W:0", "../../models/mnf/weights/mean_layer1_{}.npz".format(model))
#     save_variable_to_file("fq2_fr2_usezTrue/densemnf_1/var_W:0", "../../models/mnf/weights/var_layer1_{}.npz".format(model))


INFO:tensorflow:Restoring parameters from ../../models/mnf/mnf_log-uniform_clipped-alpha_MNIST/mnf
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/mean_W:0' shape=(5, 5, 1, 20) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/logvar_W:0' shape=(5, 5, 1, 20) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/mean_bias:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/logvar_bias:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/dropout_rates_mean:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/dropout_rates:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/var_r_aux:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/apvar_r_aux:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/var_r_auxi:0' shape=(20,) dtype=float32_ref>
<tf.Variable 'fq2_fr2_usezTrue/conv2dmnf_1/prior_var_r_p:0' shape=(25,) dtype=float32_re

In [ ]:
def plot_function(weights, subset=True, savefile=None):
    '''
    Plot heatmaps
    '''
    if subset:
        weights = weights[:50, :50]
    plt.clf()
    plt.imshow(weights)
    plt.colorbar()
    
    if savefile is not None:
        plt.savefig(savefile)

## Results on the MNIST data

In [ ]:
weights = np.load("../../models/mnf/weights/layer1_std-normal.npz")
plot_function(weights)

In [ ]:
weights = np.load("../../models/mnf/weights/layer1_cauchy_10-epochs.npz")
plot_function(weights)

In [ ]:
weights = np.load("../../models/mnf/weights/layer1_log-uniform_recent.npz")
plot_function(weights)

In [ ]:
weights = np.load("../../models/mnf/weights/layer1_log-uniform_clipped-alpha.npz")
plot_function(weights)

In [ ]:
def plot_function_2(weights):
    masked_array = np.ma.masked_where(weights == 0.0, weights)

    cmap = matplotlib.cm.spring  # Can be any colormap that you want after the cm
    cmap.set_bad(color='black')

    plt.imshow(masked_array, cmap=cmap)
    # plt.colorbar()
    plt.show()

In [ ]:
weights = np.load("../../models/mnf/weights/layer1_log-uniform_recent.npz")
weights = weights[:50, :50]
weights[(weights >= -1) & (weights <= 1)] = 0.0
plot_function_2(weights)

In [ ]:
mean_weights = np.load("../../models/mnf/weights/mean_layer1_log-uniform_clipped-alpha.npz")
var_weights = np.load("../../models/mnf/weights/var_layer1_log-uniform_clipped-alpha.npz")

# mean_weights = mean_weights[:50, :50]
# var_weights = var_weights[:50, :50]

mean_weights[(mean_weights >= -1) & (mean_weights <= 1)] = 0.0

# print(mean_weights)

count = 0
for i in range(800):
    for j in range(500):
#         if var_weights[i, j] < -14:
        if mean_weights[i, j] == var_weights[i, j]:
            count += 1
            mean_weights[i, j] = 0.0

print(count)
# plot_function_2(mean_weights)
plot_function_2(mean_weights)

In [ ]:
from mnist import MNIST
from keras.utils.np_utils import to_categorical

mnist = MNIST()
(xtrain, ytrain), (xvalid, yvalid), (xtest, ytest) = mnist.images()
xtrain, xvalid, xtest = np.transpose(xtrain, [0, 2, 3, 1]), np.transpose(xvalid, [0, 2, 3, 1]), np.transpose(xtest, [0, 2, 3, 1])
ytrain, yvalid, ytest = to_categorical(ytrain, 10), to_categorical(yvalid, 10), to_categorical(ytest, 10)

In [ ]:
weights = np.load("../../models/mnf/weights/layer1_log-uniform_recent.npz")
weights[(weights >= -1) & (weights <= 1)] = 0.0

with tf.Session() as sess:
    saver = tf.train.import_meta_graph('../../models/mnf/mnf_cauchy_10-epochs/mnf.meta')
    saver.restore(sess, tf.train.latest_checkpoint('../../models/mnf/mnf_cauchy_10-epochs/'))

    w = [v for v in tf.trainable_variables() if v.name == "fq2_fr2_usezTrue/densemnf_1/mean_W:0"][0]
    w_new = w.assign(tf.convert_to_tensor(weights))
    sess.run(w_new)


## Results on the CIFAR-10 data

In [3]:
from cifar10 import CIFAR10
from keras.utils.np_utils import to_categorical

cifar10 = CIFAR10()
(xtrain, ytrain), (xvalid, yvalid), (xtest, ytest) = cifar10.images()
xtrain, xvalid, xtest = np.transpose(xtrain, [0, 2, 3, 1]), np.transpose(xvalid, [0, 2, 3, 1]), np.transpose(xtest, [0, 2, 3, 1])
ytrain, yvalid, ytest = to_categorical(ytrain, 10), to_categorical(yvalid, 10), to_categorical(ytest, 10)

print xtrain.shape

(40000, 32, 32, 3)


In [ ]:
# weights = np.load("../../models/mnf/weights/layer1_log-uniform_10-epochs_cifar10.npz")
import cPickle as pickle
with open('../../data/cifar10/data_batch_1', 'rb') as f:
    cifar = pickle.load(f)

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
c = np.array([[1, 2], [3, 4]])

np.concatenate((a, b, c), axis=0)

In [ ]:

data = cifar['data']
print data.shape
# single_image = np.array(data[1])
# single_image_reshaped = np.transpose(np.reshape(single_image,(3, 32, 32)), (1, 2, 0))
# # print(single_image)
# # plt.imshow(single_image_reshaped)
# print single_image_reshaped.shape

In [ ]:
from mnist import MNIST
from keras.utils.np_utils import to_categorical

mnist = MNIST()
(xtrain, ytrain), (xvalid, yvalid), (xtest, ytest) = mnist.images()
print xtrain.shape
xtrain, xvalid, xtest = np.transpose(xtrain, [0, 2, 3, 1]), np.transpose(xvalid, [0, 2, 3, 1]), np.transpose(xtest, [0, 2, 3, 1])
ytrain, yvalid, ytest = to_categorical(ytrain, 10), to_categorical(yvalid, 10), to_categorical(ytest, 10)

print xtrain.shape
print ytrain.shape